In [1]:
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import confusion_matrix
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
import datetime as dt
from __future__ import absolute_import
from sklearn.metrics import confusion_matrix
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader.data import DataReader 
import matplotlib.pyplot as plt
from numpy import loadtxt, where
#%matplotlib notebook
from sklearn.model_selection import cross_val_score
import itertools as iter
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, ElasticNetCV,Ridge
from sklearn.neural_network import MLPRegressor
from sklearn import linear_model
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import ARDRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
from sklearn.linear_model import RANSACRegressor
from sklearn.svm import SVR
import warnings
warnings.filterwarnings('ignore', '.*do not.*',)
pd.options.mode.chained_assignment = None # None|'warn'|'raise'

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [3]:
def MASE(train, test, forcast):
    d = np.abs(  np.diff( train.ravel()) ).sum()/(len(train.ravel())-1)
    errors = (np.abs(test - forcast )).mean()
    return errors / d

In [4]:
def MASE_(forcast, label):
    forcast = forcast[2:-2]
    label = label[2:-2]
#    print(forcast[:5])
 #   print(forcast[-5:])
    label = label.flatten()
 #   print(label[:5])
 #   print(label[-5:])
    d = (np.abs(forcast - label)).mean()
    naive = np.abs(np.diff(label, n=1)).mean()
    return d/naive

In [5]:
end = dt.datetime.now()
start = end - dt.timedelta (days = 5 * 356)
df = DataReader("MU","iex",start, end)
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
df['year'] = df.index.year.values
df['month'] = df.index.month.values
df['day'] = df.index.day.values
df['date'] = df['year'].astype(str)+ '-' +df['month'].astype(str) + '-' + df['day'].astype(str)

5y


In [6]:
df.reset_index(drop=True)
df.head()

,open,high,low,close,volume,year,month,day,date
date,,,,,,,,,
2014-01-23,23.31,23.650,23.24,23.64,29434357,2014,1,23,2014-1-23
2014-01-24,23.50,23.650,22.91,22.92,30309956,2014,1,24,2014-1-24
2014-01-27,23.10,23.370,22.56,22.90,30040795,2014,1,27,2014-1-27
2014-01-28,22.89,23.200,22.55,23.05,27589769,2014,1,28,2014-1-28
2014-01-29,22.84,23.265,22.65,22.71,21732216,2014,1,29,2014-1-29


In [7]:
def count_momentum(dataFrame, periods, column_close='close'):
    dataFrame['Momentum'] = np.nan
    for i, (index, row) in enumerate(df.iterrows()):
        if i >= periods:
            previous_value = df.loc[df.index[i-periods], column_close]
            current_value = df.loc[df.index[i], column_close]
            final_value = (current_value - previous_value)
            dataFrame.loc[index, 'Momentum'] = final_value

    return dataFrame

In [8]:
#ROC (rate of change)	Ct/C(t−n) × 100,
def ROC(dataFrame, periods, column_close='close'):
    dataFrame['ROC'] = np.nan
    for i, (index, row) in enumerate(df.iterrows()):
        if i >= periods :
            momentum = dataFrame.loc[index,'Momentum']
            shift_value = df.loc[df.index[ i - periods ], column_close]
            ROC = momentum / shift_value
            dataFrame.loc[index, 'ROC'] = ROC * 100

    return dataFrame

In [9]:
#MA5	MA5=(∑5i=1Ct−i+1)/5
def count_moving_average(dataFrame, periods, column_close='close'):
    dataFrame['Mov_Avg'] = np.nan
    for i, (index, row) in enumerate(df.iterrows()):
        if i >= periods - 1:
            sum = 0
            for j in range(periods):
                sum += df.loc[df.index[ i - j  ], column_close]     
            ma_value = sum/periods              
            dataFrame.loc[index, 'Mov_Avg'] = ma_value

    return dataFrame
#df['Moving_Average_5'] = pd.Series(df['close'].rolling(5, min_periods=5).mean())
#df['Moving_Average_5'] = pd.Series(df['close'].rolling(5, min_periods=5).mean())

In [10]:
def count_OBV(dataFrame):
    dataFrame['change'] = [0]*(len(dataFrame)) 
    dataFrame.change[dataFrame.close > dataFrame.close.shift(1)] = +1
    dataFrame.change[dataFrame.close < dataFrame.close.shift(1)] = -1
    dataFrame['OBV'] = (dataFrame.volume * dataFrame.change).cumsum()
    #dataFrame.is_copy= False
    return dataFrame

In [11]:
def count_Stochastic_k(dataFrame):
    lowest = dataFrame['low'].rolling(window=2).min()
    highest = dataFrame['high'].rolling(window=2).max()
    dataFrame['Stochastic_k'] = (dataFrame['close'] - lowest) / (highest - lowest) * 100
    return dataFrame

In [12]:
count_OBV(df)
count_momentum(df,2)
ROC(df, 2)
count_moving_average(df,2)
count_Stochastic_k(df)
df.head(2)

,open,high,low,close,volume,year,month,day,date,change,OBV,Momentum,ROC,Mov_Avg,Stochastic_k
date,,,,,,,,,,,,,,,
2014-01-23,23.31,23.65,23.24,23.64,29434357,2014,1,23,2014-1-23,0,0,NaN,NaN,NaN,NaN
2014-01-24,23.50,23.65,22.91,22.92,30309956,2014,1,24,2014-1-24,-1,-30309956,NaN,NaN,23.28,1.351351


In [13]:
def addVectors(dataFrame,num,lst):
    for col in lst:
        for j in range(num):
            dataFrame[col +'_'+ str(j)] = np.nan
    for col in lst:
        for i, (index, row) in enumerate(dataFrame.iterrows()):
            if i >= num -1:
                for j in range(num):
                    col_name = col+'_'+ str(j)
                    indx = dataFrame.index[ i + j - ( num - 1 )]
                    number = dataFrame.loc[indx, col]
                    dataFrame.loc[index, col_name] = number
    return dataFrame

In [14]:
#Here you can test addVectors function
# You can create any number of vectors form any number of columns
test_list = ['close','Momentum']
test_df = df[['close','OBV','Momentum','ROC','Mov_Avg','Stochastic_k']]
#test_df = df[['close','OBV']]
addVectors(test_df,3,test_list)
test_df.sort_index(axis=1, inplace=True)
#test_df = test_df.dropna()
test_df.head(10)

,Momentum,Momentum_0,Momentum_1,Momentum_2,Mov_Avg,OBV,ROC,Stochastic_k,close,close_0,close_1,close_2
date,,,,,,,,,,,,
2014-01-23,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,23.64,NaN,NaN,NaN
2014-01-24,NaN,NaN,NaN,NaN,23.280,-30309956,NaN,1.351351,22.92,NaN,NaN,NaN
2014-01-27,-0.74,NaN,NaN,-0.74,22.910,-60350751,-3.130288,31.192661,22.90,23.64,22.92,22.90
2014-01-28,0.13,NaN,-0.74,0.13,22.975,-32760982,0.567190,60.975610,23.05,22.92,22.90,23.05
2014-01-29,-0.19,-0.74,0.13,-0.19,22.880,-54493198,-0.829694,22.377622,22.71,22.90,23.05,22.71
2014-01-30,0.40,0.13,-0.19,0.40,23.080,-21836992,1.735358,72.072072,23.45,23.05,22.71,23.45
2014-01-31,0.33,-0.19,0.40,0.33,23.245,-51394625,1.453104,11.111111,23.04,22.71,23.45,23.04
2014-02-03,-0.64,0.40,0.33,-0.64,22.925,-87431351,-2.729211,11.827957,22.81,23.45,23.04,22.81
2014-02-04,-0.04,0.33,-0.64,-0.04,22.905,-66035901,-0.173611,35.714286,23.00,23.04,22.81,23.00


In [15]:
filtered_df = pd.DataFrame()
filtered_df = df[['close','OBV','Momentum','ROC','Mov_Avg','Stochastic_k']]
filtered_cols = ['close','OBV','Momentum','ROC','Mov_Avg','Stochastic_k']
filtered_df = filtered_df.dropna()
filtered_df['label'] = filtered_df.close.shift(-1)

In [16]:
def make_decision(dataFrame):
    for i, (index, row) in enumerate(dataFrame.iterrows()):
              
        current_Actual = dataFrame.loc[dataFrame.index[i], 'Actual']
        prev_Actual = dataFrame.loc[dataFrame.index[i - 1 ], 'Actual']
        
        current_Forcast = dataFrame.loc[dataFrame.index[i], 'Forcast']
        prev_Forcast = dataFrame.loc[dataFrame.index[i - 1], 'Forcast']
        
        #Actual_Momentum
        act_sub = current_Actual -  prev_Actual
        dataFrame.at[index, 'Actual_Momentum'] = act_sub

        #Predicted_Momentum
        pre_sub = current_Forcast -  prev_Forcast
        dataFrame.at[index, 'Forcast_Momentum'] = pre_sub
       
        #Pi
        multi = act_sub * pre_sub
     
        if act_sub > 0:
            dataFrame.at[index, 'Actual_Decision'] = "Buy"   
        elif act_sub < 0 :
            dataFrame.at[index, 'Actual_Decision'] = "Sell"
        else:
            dataFrame.at[index, 'Actual_Decision'] = "Sell"
       
    
        if pre_sub > 0:
            dataFrame.at[index, 'Forcast_Decision'] = "Buy"   
        elif pre_sub < 0 :
            dataFrame.at[index, 'Forcast_Decision'] = "Sell"
        else:
            dataFrame.at[index, 'Forcast_Decision'] = "Sell"
            
           
        
        if multi > 0 :
            dataFrame.at[index, 'Pi'] = 1
        else:
            dataFrame.at[index, 'Pi'] = 0
        
    return 

In [17]:
def build_model(data):
    
    if 'close' not in data.columns:
        data['label'] = filtered_df['label']
        data = data.dropna()
        Y = np.array(data.label)
        #data = data.drop(data.label, axis=1)
        data = data.drop(columns=['label'])
        Y = Y.reshape(-1,1)
        close = np.array(filtered_df.close)
    else:
        data['label'] = data.close.shift(-1)
        #data = data[np.isfinite(df['label'])]
        data = data.dropna()
        Y = np.array(data.label)
        close = np.array(data.close)
    

    all_col = data.columns.values.tolist()
    X = np.array(data[all_col])

    tscv = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        close_train, close_test = close[train_index], close[test_index]
    #print("hello!!",close_test)   
    linear = RandomForestRegressor()
    linear.fit(X_train, Y_train.reshape(len(Y_train),) )
    forcast = linear.predict(X_test)
    #print(Y_test)
    return forcast, X_train, X_test, Y_train, Y_test,close_test


In [18]:
model_values = build_model(filtered_df)

In [19]:
#decision_df = pd.DataFrame(columns=['Forcast','Actual','Close'])
def create_decision_df(value):
    dataFrame = pd.DataFrame()
    dataFrame['Forcast'] = value[0]
    dataFrame['Actual'] = value[4]
    dataFrame['Close'] = value[5]
    dataFrame = dataFrame.dropna()
    return dataFrame

decision_df = create_decision_df(model_values)
make_decision(decision_df)

In [20]:
decision_df.head()

,Forcast,Actual,Close,Actual_Momentum,Forcast_Momentum,Actual_Decision,Forcast_Decision,Pi
0,44.216,44.21,43.50,8.90,8.756,Buy,Buy,1.0
1,44.745,44.90,44.21,0.69,0.529,Buy,Buy,1.0
2,44.546,44.54,44.90,-0.36,-0.199,Sell,Sell,1.0
3,44.361,44.25,44.54,-0.29,-0.185,Sell,Sell,1.0
4,46.142,46.53,44.25,2.28,1.781,Buy,Buy,1.0


In [21]:
def get_confusion_matrix(y_test, y_pred):
    TP = FP = TN = FN = 0
    for i in range(len(y_pred)): 
        if y_test[i] == y_pred[i] == 'Buy':
            TP += 1
        if y_pred[i] == 'Buy' and y_test[i]!=y_pred[i]:
            FP += 1
        if y_test[i] == y_pred[i]== 'Sell':
            TN += 1
        if y_pred[i] =='Sell'  and y_test[i] !=y_pred[i]:
            FN += 1
    data = dict()
    #data = {'True positive':TP,'False positive':FP,'False negative':FN,'True negative':TN}
    data = {'TP':TP,'FP':FP,'FN':FN,'TN':TN}
    return data


In [22]:
comparison_df = pd.DataFrame(columns=['MASE_OLD', 'MASE_NEW', 'SMAPE','Confusion_Matrix','Hit_Rate'])

In [23]:
def do_comparison(data, num, cols):
    if num > 0:
        for i in range(1, num + 1):
            #
            addVectors(data, i, cols)
            #
            output = build_model(data)
            #forcast, X_train, X_test, Y_train,
            smape_value = smape(output[4].ravel(), output[0])
            MASE_value = MASE(output[1], output[2].reshape(-1,1),output[0])
            MASE_value_ = MASE_(output[0], output[4])
            #
            dataFrame = create_decision_df(output)
            vals = make_decision(dataFrame)
            #
            hit_ratio = (dataFrame.Pi.mean() * 100)
            cm = get_confusion_matrix(dataFrame.Actual_Decision, dataFrame.Forcast_Decision)
            comparison_df.at[i, 'SMAPE'] = "{:.3f}".format(smape_value)
            comparison_df.at[i, 'MASE_OLD'] = "{:.3f}".format(MASE_value)
            comparison_df.at[i, 'MASE_NEW'] = "{:.3f}".format(MASE_value_)
            comparison_df.at[i, 'Hit_Rate'] = "{:.3f}".format(hit_ratio)
            comparison_df.at[i, 'Confusion_Matrix'] = cm
            
                 
    return 

Grade 1: You should extend your previous project solution and make use of a time window for input. Run tests on how the regression error changes and if the decision correctness is improved. Compare using sMAPE and MASE for regression and confusion matrix and hit rate for the decision.

In [42]:
#filtered_cols = ['close','OBV','Momentum','ROC','Mov_Avg','Stochastic_k']
#do_comparison(filtered_df,1,filtered_cols)
just_close = ['close']
do_comparison(filtered_df,3,just_close)

In [43]:
comparison_df.head()

,MASE_OLD,MASE_NEW,SMAPE,Confusion_Matrix,Hit_Rate
1,2.113,3.086,6.508,"{'TP': 51, 'FP': 15, 'FN': 54, 'TN': 84}",51.471
2,2.112,3.082,6.538,"{'TP': 67, 'FP': 21, 'FN': 38, 'TN': 78}",61.275
3,2.110,3.160,6.712,"{'TP': 60, 'FP': 16, 'FN': 45, 'TN': 83}",60.294


Grade + 1: You need to do feature selection, you will do a brute force search for which feature or combination of features that provide you with the best result (regression and decision error).
 You should evaluate at least 5 feature inputs but no more than 8!


In [44]:
columns =  ['MASE_OLD', 'MASE_NEW','SMAPE','confusion_matrix','cloumns_used']
brute_force_df  = pd.DataFrame(columns = columns)

In [45]:
def brute_force(df,clos):
    comb = (iter.combinations(filtered_cols, l) for l in range(len(filtered_cols) + 1))
    ls = list(iter.chain.from_iterable(comb))
    for i,j in enumerate(ls):
        if i > 0:
            arr = []
            for k,l in enumerate(j):
                arr.append(l)
            df2 = pd.DataFrame(df[arr])
            df2 = df2.dropna()
            output = build_model(df2)
            dataFrame = create_decision_df(output)
            vals = make_decision(dataFrame)
            
            #forcast, X_train, X_test, Y_train,
            smape_value = smape(output[4].ravel(), output[0])
            MASE_value = MASE(output[1], output[2].reshape(-1,1),output[0])
            MASE_value_ = MASE_(output[0], output[4])
            
            #
    
            brute_force_df.at[i, 'SMAPE'] = "{:.3f}".format(smape_value)
            brute_force_df.at[i, 'MASE_OLD'] = "{:.3f}".format(MASE_value)
            brute_force_df.at[i, 'MASE_NEW'] = "{:.3f}".format(MASE_value_)
           
            hit_ratio = (dataFrame.Pi.mean() * 100)
            cm = get_confusion_matrix(dataFrame.Actual_Decision, dataFrame.Forcast_Decision)
            brute_force_df.at[i, 'Hit_ratio'] = "{:.1f}".format(hit_ratio)
            brute_force_df.at[i, 'confusion_matrix'] = cm
            brute_force_df.at[i, 'cloumns_used'] = arr
            #print('The confusion_matrix cloumns', arr , ' ' , hr_cm[1].ravel()) 
            
    return 

In [46]:
brute_force(filtered_df,filtered_cols)

In [47]:
brute_force_df.head()
brute_force_df.head(100)

,MASE_OLD,MASE_NEW,SMAPE,confusion_matrix,cloumns_used,Hit_ratio
1,30.397,2.763,5.800,"{'TP': 43, 'FP': 3, 'FN': 62, 'TN': 96}",[close],43.6
2,44.387,5.928,13.715,"{'TP': 3, 'FP': 1, 'FN': 102, 'TN': 98}",[OBV],2.0
3,42.440,18.151,52.637,"{'TP': 45, 'FP': 52, 'FN': 60, 'TN': 47}",[Momentum],45.1
4,8.659,21.683,66.573,"{'TP': 55, 'FP': 49, 'FN': 50, 'TN': 50}",[ROC],51.0
5,20.245,3.752,8.244,"{'TP': 20, 'FP': 17, 'FN': 85, 'TN': 82}",[Mov_Avg],20.1
6,1.131,22.109,67.382,"{'TP': 57, 'FP': 43, 'FN': 48, 'TN': 56}",[Stochastic_k],54.4
7,2.113,2.927,6.186,"{'TP': 46, 'FP': 1, 'FN': 59, 'TN': 98}","[close, OBV]",46.1
8,1.216,2.984,6.283,"{'TP': 58, 'FP': 14, 'FN': 47, 'TN': 85}","[close, Momentum]",58.3
9,1.222,3.100,6.544,"{'TP': 59, 'FP': 17, 'FN': 46, 'TN': 82}","[close, ROC]",57.4
10,27.392,2.845,5.978,"{'TP': 46, 'FP': 1, 'FN': 59, 'TN': 98}","[close, Mov_Avg]",45.1


In [48]:
#JUST A TEST
f_df = df[['OBV']]
#filtered_cols = ['close','OBV','Momentum','ROC','Mov_Avg','Stochastic_k']
V_L = build_model(f_df)
a = MASE(V_L[1], V_L[2].reshape(-1,1),V_L[0])
print(a)

44.387382285228085


Grade + 2: Rerun your models on intraday data, determine the most profitable resolution (1m, 5m, 10m, 1h, 1d) for your trading algorithm (P/L account).

In [ ]:
# Please remember to change the EURAUD_20110101-20141231.csv
import datetime as dt
import pandas as pd
csvfile = pd.read_csv('EURAUD_20110101-20141231.csv')
#csvfile = pd.read_csv('daydata.csv')
csvfile['close'] = csvfile['Close']
csvfile['low'] = csvfile['Low']
csvfile['high'] = csvfile['High']
csvfile['volume'] = csvfile['Volume']
#csvfile = csvfile.drop(columns=['Close'])
csvfile['hour'] = pd.DatetimeIndex(csvfile['Timestamp']).hour
csvfile['minute'] = pd.DatetimeIndex(csvfile['Timestamp']).minute
csvfile['second'] = pd.DatetimeIndex(csvfile['Timestamp']).second

In [26]:
count_OBV(csvfile)
csvfile['Mov_Avg'] = pd.Series(csvfile['close'].rolling(5, min_periods=5).mean())
csvfile['Momentum'] = pd.Series(csvfile['close'].diff(4))
csvfile['ROC'] = csvfile.close.pct_change(4)
count_Stochastic_k(csvfile)
csvfile = csvfile.dropna()

In [27]:
csvfile.head()

,Date,Timestamp,Open,High,Low,Close,Volume,close,low,high,volume,hour,minute,second,change,OBV,Mov_Avg,Momentum,ROC,Stochastic_k
4,20110102,22:04:00,1.30671,1.30680,1.30671,1.30680,34.660000,1.30680,1.30671,1.30680,34.660000,22,4,0,1,66.050000,1.306610,-0.00027,-0.000207,100.000000
5,20110102,22:05:00,1.30680,1.30680,1.30680,1.30680,1.730000,1.30680,1.30680,1.30680,1.730000,22,5,0,0,66.050000,1.306556,0.00092,0.000705,100.000000
6,20110102,22:06:00,1.30684,1.30728,1.30679,1.30709,96.710001,1.30709,1.30679,1.30728,96.710001,22,6,0,1,162.760001,1.306798,0.00050,0.000383,61.224490
7,20110102,22:07:00,1.30681,1.30681,1.30613,1.30615,119.400000,1.30615,1.30613,1.30681,119.400000,22,7,0,-1,43.360001,1.306710,-0.00056,-0.000429,1.739130
8,20110102,22:08:00,1.30613,1.30616,1.30613,1.30616,48.770000,1.30616,1.30613,1.30616,48.770000,22,8,0,1,92.130000,1.306600,-0.00064,-0.000490,4.411765


In [28]:
csvfile = csvfile[['close','OBV', 'Momentum', 'ROC','Mov_Avg','Stochastic_k']]

In [29]:
def three_decisions(dataFrame):
    
    for i, (index, row) in enumerate(dataFrame.iterrows()):
        current_actual = dataFrame.loc[dataFrame.index[i], 'Actual']
        prev_actual = dataFrame.loc[dataFrame.index[i - 1 ], 'Actual']
        act_sub = current_actual -  prev_actual
        dataFrame.at[index, 'Actual_Momentum'] = act_sub
        prev_mom_act = dataFrame.loc[dataFrame.index[i-1], 'Actual_Momentum']
        
        if act_sub > 0 and prev_mom_act < 0:
            dataFrame.at[index, 'Actual_Decision'] = "Buy"   
        if act_sub >= 0 and prev_mom_act >= 0:
            dataFrame.at[index, 'Actual_Decision'] = "Hold"
            #dataFrame.at[index, 'Actual_Cost'] = current_Actual
        if act_sub < 0 and prev_mom_act > 0:
            dataFrame.at[index, 'Actual_Decision'] = "Sell"
        if act_sub <= 0 and prev_mom_act <= 0:
            dataFrame.at[index, 'Actual_Decision'] = "Hold"   
            
    return 

In [30]:
def count_profit_loss(dataFrame):
    dataFrame['PL'] = 0.0
    dataFrame['Cost'] = 0.0
    for i, (index, row) in enumerate(dataFrame.iterrows()):
        current_AD = dataFrame.loc[dataFrame.index[i], 'Actual_Decision']
        current_A = dataFrame.loc[dataFrame.index[i], 'Close'] 
        prev_pl  = dataFrame.loc[dataFrame.index[i-1], 'PL']
        if current_AD == 'Buy':
            dataFrame.at[index, 'Cost'] = current_A 
            dataFrame.at[index, 'PL'] = prev_pl 
        elif current_AD == 'Hold':
             dataFrame.at[index, 'PL'] = prev_pl 
        elif current_AD == 'Sell':
            prev_pl  = dataFrame.loc[dataFrame.index[i-1], 'PL']
            curr_val = dataFrame[dataFrame.index <= index].iloc[::-1].Actual.values[:1][0]
            d = dataFrame[dataFrame.index < index].iloc[::-1]
            for j, (indx, rw) in enumerate(d.iterrows()):
                act_dec = d.loc[d.index[j], 'Actual_Decision']
                if  act_dec =="Hold":
                    continue
                if act_dec =='Buy':
                    act_val = d.loc[d.index[j], 'Close']
                    if prev_pl == np.nan:
                        dataFrame.at[index, 'PL'] = curr_val - act_val 
                    elif prev_pl != np.nan:
                        
                        output = (curr_val - act_val) + prev_pl
                        dataFrame.at[index, 'PL'] = output
                    break
                else:
                    break
    return dataFrame


In [31]:
csvfile_df = build_model(csvfile)
three_decisions_df = create_decision_df(csvfile_df)
three_decisions(three_decisions_df)
count_profit_loss(three_decisions_df)
three_decisions_df['Profit/Loss_1m'] = pd.Series(three_decisions_df['PL'].rolling(1).max())
three_decisions_df['Profit/Loss_10m'] = pd.Series(three_decisions_df['PL'].rolling(10).max())
three_decisions_df['Profit/Loss_1h'] = pd.Series(three_decisions_df['PL'].rolling(60).max())
three_decisions_df['Profit/Loss_1d'] = pd.Series(three_decisions_df['PL'].rolling(1440).max())
#three_decisions_df = three_decisions_df.dropna()
three_decisions_df.tail(10)

,Forcast,Actual,Close,Actual_Momentum,Actual_Decision,PL,Cost,Profit/Loss_1m,Profit/Loss_10m,Profit/Loss_1h,Profit/Loss_1d
260,1.311898,1.31414,1.31428,-0.00014,Sell,0.00360,0.00000,0.00360,0.00360,0.00362,NaN
261,1.311898,1.31424,1.31414,0.00010,Buy,0.00360,1.31414,0.00360,0.00360,0.00362,NaN
262,1.311898,1.31448,1.31424,0.00024,Hold,0.00360,0.00000,0.00360,0.00360,0.00362,NaN
263,1.311898,1.31470,1.31448,0.00022,Hold,0.00360,0.00000,0.00360,0.00360,0.00362,NaN
264,1.311898,1.31476,1.31470,0.00006,Hold,0.00360,0.00000,0.00360,0.00360,0.00362,NaN
265,1.311898,1.31483,1.31476,0.00007,Hold,0.00360,0.00000,0.00360,0.00360,0.00362,NaN
266,1.311898,1.31460,1.31483,-0.00023,Sell,0.00406,0.00000,0.00406,0.00406,0.00406,NaN
267,1.311898,1.31469,1.31460,0.00009,Buy,0.00406,1.31460,0.00406,0.00406,0.00406,NaN
268,1.311898,1.31433,1.31469,-0.00036,Sell,0.00379,0.00000,0.00379,0.00406,0.00406,NaN
269,1.311898,1.31442,1.31433,0.00009,Buy,0.00379,1.31433,0.00379,0.00406,0.00406,NaN


Grade + 1: Setup a ML pipeline based on sklearn. You should evaluate if an ensemble of 20 regression models give you improved errors (reg & decision) and if the model return improves. State clearly how your ensemble works.

In [32]:
regression_names = ['DecisionTreeRegressor', 'ElasticNet','Ridge','Lasso','LassoLars',
                   'BayesianRidge','SGDRegressor','TheilSenRegressor',
                   'HuberRegressor','LinearRegression',
                   'RandomForestRegressor','ExtraTreesRegressor','AdaBoostRegressor',
                   'GradientBoostingRegressor','BaggingRegressor',
                   'ARDRegression','RANSACRegressor','SVR','PassiveAggressiveRegressor','tree.DecisionTreeRegressor']

regression_list = [DecisionTreeRegressor, ElasticNet,Ridge,Lasso,LassoLars,
                   BayesianRidge,SGDRegressor,TheilSenRegressor,
                   HuberRegressor,LinearRegression,
                   RandomForestRegressor,ExtraTreesRegressor,AdaBoostRegressor,
                   GradientBoostingRegressor,BaggingRegressor,
                   ARDRegression,RANSACRegressor,SVR,PassiveAggressiveRegressor,tree.DecisionTreeRegressor]

In [33]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#class_df = pd.DataFrame()
columns =  ['Model','MASE_OLD', 'MASE_NEW', 'SMAPE','confusion_matrix','Hit_ratio','Cross_val_score', 'Mean','Standard_Deviation*2']
ensemble_df  = pd.DataFrame(columns = columns)
classify_df  = pd.DataFrame( columns=['xtest'])

In [34]:
def build_pipline(data, reg_list,reg_names):
  
    data['label'] = data.close.shift(-1)
    data = data.dropna()
    Y = np.array(data.label)
    X = np.array(data[['close','OBV','Momentum','ROC','Mov_Avg','Stochastic_k']])
    
    tscv = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
  
    classify_df['xtest'] = X_test.tolist()
    
    pipeline_list = []
    class_list = []
    accuracy = []
    for rg_model in reg_list: 
        
        rg_model = Pipeline([('scl', StandardScaler()),
                ('pca', PCA(n_components=2)),
                ('clf', rg_model())])
        pipeline_list.append(rg_model)
    
    for pipe in pipeline_list:
        pipe.fit(X_train, y_train)
        
        
        scores = cross_val_score(pipe, X, Y, cv=5)
        formattedList = ["%.2f" % member for member in scores]
        class_list.append(formattedList)
        mn = "{:.2f}".format(scores.mean())
        std = "{:.2f}".format(scores.std()* 2) 
        accuracy.append((mn, std ))

    for index, value in enumerate(pipeline_list):
        forcast = value.predict(X_test)

        #class_df[regression_names[index]] = forcast.tolist()
        
       
        dataF = pd.DataFrame({'Actual':y_test ,'Forcast':forcast })
        
        vals = make_decision(dataF)
        hit_ratio = (dataF.Pi.mean() * 100)
        cm = get_confusion_matrix(dataF.Actual_Decision, dataF.Forcast_Decision)
        classify_df[regression_names[index]] = dataF.Pi
        smape_value = smape(y_test.ravel(), forcast)
        #MASE_value = MASE(X_train, X_test.reshape(-1,1),forcast)
        MASE_value = MASE(X_train.reshape(-1,1), X_test.reshape(-1,1),forcast)
        MASE_value_ = MASE_(forcast, y_test)
        #MASE_value = MASE_1(forcast, y_test)
    
        ensemble_df.at[index, 'SMAPE'] = "{:.2f}".format(smape_value)
        #ensemble_df.at[index, 'MASE'] = "{:.9f}".format(MASE_value)
        ensemble_df.at[index, 'MASE_OLD'] = "{:.9f}".format(MASE_value)
        ensemble_df.at[index, 'MASE_NEW'] = "{:.9f}".format(MASE_value_)
        ensemble_df.at[index, 'Hit_ratio'] = "{:.2f}".format(hit_ratio)
        ensemble_df.at[index, 'confusion_matrix'] = cm
        ensemble_df.at[index, 'Model'] = reg_names[index]
        ensemble_df.at[index, 'Cross_val_score'] = class_list[index]
        ensemble_df.at[index, 'Mean'] = accuracy[index][0]
        ensemble_df.at[index, 'Standard_Deviation*2'] = accuracy[index][1]
        
    
    return 

build_pipline(df,regression_list,regression_names)
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
ensemble_df.head()

,Model,MASE_OLD,MASE_NEW,SMAPE,confusion_matrix,Hit_ratio,Cross_val_score,Mean,Standard_Deviation*2
0,DecisionTreeRegressor,2.113317917,4.741665181,10.84,"{'TP': 33, 'FP': 31, 'FN': 72, 'TN': 68}",30.88,"[0.84, 0.95, 0.17, 0.91, 0.41]",0.66,0.62
1,ElasticNet,2.113317921,2.582379745,5.86,"{'TP': 57, 'FP': 49, 'FN': 48, 'TN': 50}",52.45,"[0.89, 0.85, -0.22, 0.91, 0.71]",0.63,0.86
2,Ridge,2.113317931,2.792844517,6.58,"{'TP': 56, 'FP': 50, 'FN': 49, 'TN': 49}",51.47,"[0.93, 0.97, 0.94, 0.96, 0.76]",0.92,0.15
3,Lasso,2.113317925,1.861074821,4.49,"{'TP': 55, 'FP': 50, 'FN': 50, 'TN': 49}",50.98,"[0.91, 0.95, 0.68, 0.95, 0.87]",0.87,0.20
4,LassoLars,2.113317951,21.752774531,64.09,"{'TP': 0, 'FP': 0, 'FN': 105, 'TN': 99}",0.00,"[-0.04, -2.32, -30.48, -0.20, -11.89]",-8.98,23.19


Grade + 2: Based on a ML pipeline in sklearn do a classification as the metamodel (the ensemble model) based on a set of regression models.

In [35]:
classify_df.head()

,xtest,DecisionTreeRegressor,ElasticNet,Ridge,Lasso,LassoLars,BayesianRidge,SGDRegressor,TheilSenRegressor,HuberRegressor,...,RandomForestRegressor,ExtraTreesRegressor,AdaBoostRegressor,GradientBoostingRegressor,BaggingRegressor,ARDRegression,RANSACRegressor,SVR,PassiveAggressiveRegressor,tree.DecisionTreeRegressor
0,"[43.5, 708517136.0, 0.8699999999999974, 2.0408...",1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,"[44.21, 757921729.0, 0.759999999999998, 1.7491...",0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
2,"[44.9, 794137626.0, 1.3999999999999986, 3.2183...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"[44.54, 762518888.0, 0.3299999999999983, 0.746...",1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,"[44.25, 738159310.0, -0.6499999999999986, -1.4...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
classification_df = pd.DataFrame()
def do_classification(data):
    data.reset_index(drop=True, inplace=True)
    data = data.dropna()
    X = np.array(data.xtest)
    for i in regression_names:
        Y = np.array(data[i])
    
        tscv = TimeSeriesSplit(n_splits=5)
        for train_index, test_index in tscv.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]
        XT= []
        for j in X_train:
            XT.append(np.array(j))
        clf = tree.DecisionTreeClassifier()
        clf.fit(XT, y_train)
        classification = clf.predict(XT)
        classification_df[i] = classification
    return 
do_classification(classify_df)

In [37]:
classification_df.head()

,DecisionTreeRegressor,ElasticNet,Ridge,Lasso,LassoLars,BayesianRidge,SGDRegressor,TheilSenRegressor,HuberRegressor,LinearRegression,RandomForestRegressor,ExtraTreesRegressor,AdaBoostRegressor,GradientBoostingRegressor,BaggingRegressor,ARDRegression,RANSACRegressor,SVR,PassiveAggressiveRegressor,tree.DecisionTreeRegressor
0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
classify_df.head()

,xtest,DecisionTreeRegressor,ElasticNet,Ridge,Lasso,LassoLars,BayesianRidge,SGDRegressor,TheilSenRegressor,HuberRegressor,...,RandomForestRegressor,ExtraTreesRegressor,AdaBoostRegressor,GradientBoostingRegressor,BaggingRegressor,ARDRegression,RANSACRegressor,SVR,PassiveAggressiveRegressor,tree.DecisionTreeRegressor
0,"[43.5, 708517136.0, 0.8699999999999974, 2.0408...",1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,"[44.21, 757921729.0, 0.759999999999998, 1.7491...",0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
2,"[44.9, 794137626.0, 1.3999999999999986, 3.2183...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"[44.54, 762518888.0, 0.3299999999999983, 0.746...",1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,"[44.25, 738159310.0, -0.6499999999999986, -1.4...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Grade + 1: Make use of (deep) recurrent neural network, e.g. Long-Short Term Memory (LSTM) for doing the regression.

In [39]:
input_df = pd.DataFrame()
input_df['close'] = df['close']
#input_df['ROC'] = df['ROC']
#input_df['Momentum'] = df['Momentum']

In [40]:
import keras
import tensorflow as tf
from keras.models import Sequential  
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
in_out_neurons= 1
hidden_neurons= 300
def perform_sequential(in_out_neurons, hidden_neurons):
    model = Sequential()
    model.add(LSTM(hidden_neurons, return_sequences=False,
               input_shape=(None, in_out_neurons)
                #input_shape=(1024, 2)   
                
                  ))
    model.add(Dense(in_out_neurons, input_dim=hidden_neurons))  
    model.add(Activation("linear"))  
    model.compile(loss="mean_squared_error", optimizer="rmsprop")
    return model

In [41]:
def fit_predict(data, model):
    data['label'] = data.close.shift(-1)
    data = data.dropna()
    Y = np.array(data.label).reshape(-1,1)
    data = data.drop(columns=['label'])
    all_col = data.columns.values.tolist()
    X = np.array(data[all_col])
    
    tscv = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
    
    X_train = X_train.reshape(-1, X_train.shape[1], 1)
    X_test = X_test.reshape(-1, X_test.shape[1], 1)
    model.fit(X_train, y_train, batch_size=450, epochs=10, validation_split=0.05)
    predicted = model.predict(X_test)  
    rmse = np.sqrt(((predicted - y_test) ** 2).mean(axis=0))
    return rmse
model = perform_sequential(in_out_neurons, hidden_neurons)
fit_predict(filtered_df,model)

Train on 970 samples, validate on 52 samples
Epoch 1/10
970/970 [==============================] - 4s 4ms/step - loss: 609.4602 - val_loss: 1405.1725
Epoch 2/10
970/970 [==============================] - 1s 662us/step - loss: 378.5687 - val_loss: 898.0254
Epoch 3/10
970/970 [==============================] - 1s 669us/step - loss: 176.0293 - val_loss: 587.2704
Epoch 4/10
970/970 [==============================] - 1s 674us/step - loss: 96.1209 - val_loss: 447.1616
Epoch 5/10
970/970 [==============================] - 1s 733us/step - loss: 64.1076 - val_loss: 377.9991
Epoch 6/10
970/970 [==============================] - 1s 686us/step - loss: 48.0499 - val_loss: 333.7035
Epoch 7/10
970/970 [==============================] - 1s 761us/step - loss: 38.8349 - val_loss: 300.2736
Epoch 8/10
970/970 [==============================] - 1s 919us/step - loss: 36.5734 - val_loss: 272.2363
Epoch 9/10
970/970 [==============================] - 1s 955us/step - loss: 33.7407 - val_loss: 255.0980
Epoch 10

array([22.81175346])